In [1]:
import numpy as np
import pandas as pd

In [2]:
scraped = pd.read_pickle('data/scraped/scraped_with_decs.pickle')
scraped.head()

,name,image_path,price,url,raw_html,URL_name,description
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,Dom Pérignon Vintage 2008 has been distinguish...
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Louis-Roederer-Cristal-Brut-with-Two-...,Gift pack containing one bottle of 2008 Louis ...
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Laurent-Perrier-Cuvee-Rose/10521,The Cuvée Rosé from Laurent-Perrier is the mos...
3,Piper-Heidsieck Cuvee Brut in Travel Case with...,/product/images/fl_progressive/aqi87aqobd3zc56...,79.99,/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,#95 Wine Spectator Top 100 of 2019The Brut rep...
4,Clarendon Hills Astralis Syrah 2011,/product/images/fl_progressive/qajrqr4d6ttn2pf...,149.99,/product/Clarendon-Hills-Astralis-Syrah-2011/5...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Clarendon-Hills-Astralis-Syrah-2011/5...,"The pride of our portfolio, Clarendon Hills As..."


In [3]:
name_desc_txt = scraped[['name','description']]

In [4]:
name_desc_txt['start_token'] = "<|startoftext|>"
name_desc_txt['end_token'] = "<|endoftext|>"
name_desc_txt['prompt_token'] = "[prompt]"
name_desc_txt['response_token'] = "[response]"

name_desc_txt = name_desc_txt[[
    'start_token', 
    'prompt_token', 
    'name', 
    'response_token', 
    'description'
    ]]

c:\users\david\documents\github\this-wine-does-not-exist\.venv_w10\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\david\documents\github\this-wine-does-not-exist\.venv_w10\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\david\documents\github\this-wine-does-not-exist\.venv_w10\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is tryin

In [5]:
name_desc_txt.head()

,start_token,prompt_token,name,response_token,description
0,<|startoftext|>,[prompt],Dom Perignon Lenny Kravitz Limited Edition wit...,[response],Dom Pérignon Vintage 2008 has been distinguish...
1,<|startoftext|>,[prompt],Louis Roederer Cristal Brut with Two Flutes an...,[response],Gift pack containing one bottle of 2008 Louis ...
2,<|startoftext|>,[prompt],Laurent-Perrier Cuvee Rose,[response],The Cuvée Rosé from Laurent-Perrier is the mos...
3,<|startoftext|>,[prompt],Piper-Heidsieck Cuvee Brut in Travel Case with...,[response],#95 Wine Spectator Top 100 of 2019The Brut rep...
4,<|startoftext|>,[prompt],Clarendon Hills Astralis Syrah 2011,[response],"The pride of our portfolio, Clarendon Hills As..."


In [7]:
!pip install scikit-learn

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(name_desc_txt, test_size=0.2, random_state=0)
print(train.shape)
print(test.shape)

(100629, 5)
(25158, 5)


In [9]:
name_desc_txt.to_csv(
    "data/scraped/name_desc_nlp_ready.txt", 
    sep='\t',
    index=False,
    header=None)

train.to_csv(
    "data/scraped/name_desc_nlp_ready_train.txt", 
    sep='\t',
    index=False,
    header=None)

test.to_csv(
    "data/scraped/name_desc_nlp_ready_test.txt", 
    sep='\t',
    index=False,
    header=None)